In [1]:
import yaml

with open('config/config2.yaml', 'r') as file:
    config = yaml.safe_load(file)
config

{'llm': {'api_type': 'zhipuai',
  'api_key': 'd87bb8dbfc854c3dbc3edea867ca033e.3qCM41HN3pR7XG4T',
  'model': 'glm-4-flash',
  'base_url': 'https://open.bigmodel.cn/api/paas/v4/chat/completions',
  'repair_llm_output': True},
 'embedding': {'api_type': 'zhipuai',
  'api_key': 'd87bb8dbfc854c3dbc3edea867ca033e.3qCM41HN3pR7XG4T',
  'base_url': 'https://open.bigmodel.cn/api/paas/v4/embeddings',
  'model': 'embedding-2'},
 'browser': {'engine': 'selenium', 'browser_type': 'chrome'}}

In [2]:
from openai import OpenAI
client = OpenAI(
    api_key="d87bb8dbfc854c3dbc3edea867ca033e.3qCM41HN3pR7XG4T",
    base_url="https://open.bigmodel.cn/api/paas/v4/"
) 

In [5]:
import os
from dotenv import load_dotenv

# 加载环境变量
load_dotenv()
# 从环境变量中读取api_key

from openai import OpenAI
from datetime import datetime
def zhipu_web_search_tool(query: str) -> str:
    """
    使用智谱AI的GLM-4模型进行联网搜索，返回搜索结果的字符串。
    
    参数:
    - query: 搜索关键词

    返回:
    - 搜索结果的字符串形式
    """
    
    with open('config/config2.yaml', 'r') as file:
        config = yaml.safe_load(file)
    config['llm']['api_key']
    # 初始化客户端
    client = OpenAI(
        api_key=config['llm']['api_key'],
        base_url="https://open.bigmodel.cn/api/paas/v4/"
    ) 
    success = False

    # 获取当前日期
    current_date = datetime.now().strftime("%Y-%m-%d")

    print("current_date:", current_date)
    
    # 设置工具
    tools = [{
        "type": "web_search",
        "web_search": {
            "enable": True,
            "search_result": True
        }
    }]

    # 系统提示模板，包含时间信息
    system_prompt = f"""你是一个具备网络访问能力的智能助手，在适当情况下，优先使用网络信息（参考信息）来回答，
    以确保用户得到最新、准确的帮助。当前日期是 {current_date}。"""
        
    # 构建消息
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": query}
    ]
        
    # 调用API
	# 因为智谱返回的结果有时候没有web_search，所以我们可以让它反复生成，直到生成web_search为止
    while success == False:
        response = client.chat.completions.create(
            model="glm-4-flash",
            messages=messages,
            tools=tools
        )
        try:
            rst = response.web_search
            success = True
        except:
            pass
        
    
    # 返回结果
    return rst

In [6]:
rsp = zhipu_web_search_tool("最近三个月有哪些AI黑客松？")
print(rsp)

current_date: 2025-02-19
[{'content': '吴说获悉，Solana 宣布首届全球线上 AI 黑客松正式启动，将于 12 月 10 日至 23 日举行，奖金总额超 18.5 万美元，设立六大赛道聚焦 AI 代理开发及基础设施建设等创新领域。主赛道由 a16z crypto 和 Solana Foundation 支持，奖金最高 3 万美元，其余赛道涵盖 AI 基础设施、代币工具、自治聊天代理、Social/影响力者代理等，每个赛道由行业合作伙伴提供支持。参赛无需注册，作品提交将于 12 月 15 日开始。Solana 提供团队匹配、导师指导和资源支持，激励开发者构建基于 Solana 的 AI 创新应用。\n（转自：吴说）', 'icon': '', 'link': 'https://finance.sina.com.cn/blockchain/roll/2024-12-11/doc-incyzzcx0976575.shtml', 'media': '新浪财经', 'refer': 'ref_1', 'title': 'Solana 宣布首届全球线上 AI 黑客松正式启动'}, {'content': '本文来自：SendAI 负责人 Yash Agarwal\n编译Odaily星球日报\n译者Azuma\n编者按：由 SendAI 主办的 Solana AI 黑客松启动已有一定时日，但由于 AI 叙事的持续升温，社区对于相关项目的追捧热度丝毫不减。\n链上分析研究机构 Pine Analytics 昨日曾统计表示，由该场黑客松参与项目创建或直接关联的代币已多达 65 个，总市值为 4.82 亿美元，平均市值为 750 万美元，市值中位数为 170 万美元；有 13 个代币的市值超过 1000 万美元， 42 个代币的市值超过 100 万美元。\n然而，在社区情绪如此激昂的情况下，SendAI 创始人 Yash Agarwal 今日却发布了一篇长文抨击当前的炒作乱象 —— Yash 强调过去一周黑客松主页仅有 4 万次访问，但随随便便一个代币都能速通 5000 万市值SendAI 本想构建一个助推 Solana 生态 AI 发展的平台，但现在却成为了堕落者们的赌场”。在 Yash 来看，当前的泡沫会造成开发者趋向于盲目、短时，不利于 

智谱搜索函数封装成metagpt

In [7]:
from __future__ import annotations

import asyncio
import json
from concurrent import futures
from typing import Literal, Optional
from pydantic import BaseModel, ConfigDict
class ZhipuAPIWrapper(BaseModel):
    model_config = ConfigDict(arbitrary_types_allowed=True)

    loop: Optional[asyncio.AbstractEventLoop] = None
    executor: Optional[futures.Executor] = None

    async def run(
        self,
        query: str,
        max_results: int = 5,
        as_string: bool = True,
    ) -> str | list[dict]:
        """Return the results of a Zhipu search using the official Google API

        Args:
            query: The search query.
            max_results: The number of results to return.
            as_string: A boolean flag to determine the return type of the results. If True, the function will
                return a formatted string with the search results. If False, it will return a list of dictionaries
                containing detailed information about each search result.

        Returns:
            The results of the search.
        """
        loop = self.loop or asyncio.get_event_loop()
        future = loop.run_in_executor(
            self.executor,
            self._search_from_zhipu,
            query,
            max_results,
        )
        search_results = await future
        if as_string:
            return json.dumps(search_results, ensure_ascii=False)
        return search_results

    def _search_from_zhipu(self, query: str, max_results: int):
        return [
            {"link": i["link"], "snippet": i["content"], "title": i["title"]}
            for (_, i) in zip(range(max_results), zhipu_web_search_tool(query))
        ]

In [8]:
run_func = ZhipuAPIWrapper().run
found = await run_func("上海最近三个月有哪些AI会议？", max_results=3, as_string=False)
print(len(found))

current_date: 2025-02-19
3


In [2]:
from __future__ import annotations

import asyncio
from typing import Any, Callable, Optional, Union
# from metagpt.logs import logger
# from metagpt.config2 import config
from pydantic import TypeAdapter, model_validator
from metagpt.actions import Action
from metagpt.tools.search_engine import SearchEngine
from metagpt.utils.common import OutputParser
from metagpt.utils.text import generate_prompt_chunk, reduce_message_length

LANG_PROMPT = "请用{language}回答."

RESEARCH_BASE_SYSTEM = """你是一个专注于批判性思维的人工智能研究助理。你的唯一目标是根据给定的文本，撰写出结构严谨、客观公正、广受好评且文笔优美的报告。"""

SEARCH_TOPIC_PROMPT:str = """你是一个研究助理，请你为你的研究课题{topic}提供最多2个必要的关键词用于网络搜索。你的回答必须是JSON格式，例如：["关键词1", "关键词2"]。"""

SUMMARIZE_SEARCH_PROMPT = """### 要求
与你的研究课题相关的关键词及搜索结果会在“搜索结果信息”部分显示。
### 搜索结果信息
{search_results}
根据搜索结果，提供最多 {decomposition_nums} 个与你的研究课题相关的查询。
请以以下JSON格式回应：["query1", "query2", "query3", ...]。除了python的list形式的JSON格式不要输出任何其他内容。
"""

COLLECT_AND_RANKURLS_PROMPT = """### 主题
{topic}
### 查询
{query}
### 在线搜索结果
{results}
### 要求
请移除与查询或主题无关的搜索结果。
然后，根据链接的可信度对剩余的搜索结果进行排序。
如果两个结果的可信度相同，则根据相关性优先级排序。
提供排序后结果的索引，采用JSON格式，例如 [0, 1, 3, 4, ...]，不包含其他文字。
"""

class CollectLinks(Action):
    """Action class to collect links from a search engine."""

    name: str = "CollectLinks"
    i_context: Optional[str] = None
    desc: str = "Collect links from a search engine."
    search_func: Optional[Any] = None
    search_engine: Optional[SearchEngine] = None
    rank_func: Optional[Callable[[list[str]], None]] = None

    @model_validator(mode="after")
    def validate_engine_and_run_func(self):
        if self.search_engine is None:
            self.search_engine = SearchEngine.from_search_func(ZhipuAPIWrapper().run)
        return self

    async def run(
        self,
        topic: str,
        decomposition_nums: int = 4,
        url_per_query: int = 4,
        system_text: str | None = None,
    ) -> dict[str, list[str]]:
        """Run the action to collect links.

        Args:
            topic: The research topic.
            decomposition_nums: The number of search questions to generate.
            url_per_query: The number of URLs to collect per search question.
            system_text: The system text.

        Returns:
            A dictionary containing the search questions as keys and the collected URLs as values.
        """
        system_text = system_text if system_text else RESEARCH_TOPIC_SYSTEM.format(topic=topic)
        keywords = await self._aask(SEARCH_TOPIC_PROMPT.format(topic=topic))
        try:
            keywords = OutputParser.extract_struct(keywords, list)
            keywords = TypeAdapter(list[str]).validate_python(keywords)
        except Exception as e:
            logger.exception(f"fail to get keywords related to the research topic '{topic}' for {e}")
            keywords = [topic]
        results = await asyncio.gather(*(self.search_engine.run(i, as_string=False) for i in keywords))

        def gen_msg():
            while True:
                search_results = "\n".join(
                    f"#### Keyword: {i}\n Search Result: {j}\n" for (i, j) in zip(keywords, results)
                )
                prompt = SUMMARIZE_SEARCH_PROMPT.format(
                    decomposition_nums=decomposition_nums, search_results=search_results
                )
                yield prompt
                remove = max(results, key=len)
                remove.pop()
                if len(remove) == 0:
                    break

        model_name = 'gpt-4'
        prompt = reduce_message_length(gen_msg(), model_name, system_text, config.llm.max_token)
        logger.debug(prompt)
        queries = await self._aask(prompt, [system_text])
        try:
            queries = OutputParser.extract_struct(queries, list)
            queries = TypeAdapter(list[str]).validate_python(queries)
        except Exception as e:
            logger.exception(f"fail to break down the research question due to {e}")
            queries = keywords
        ret = {}
        for query in queries:
            ret[query] = await self._search_and_rank_urls(topic, query, url_per_query)
        return ret

    async def _search_and_rank_urls(self, topic: str, query: str, num_results: int = 2) -> list[str]:
        """Search and rank URLs based on a query.

        Args:
            topic: The research topic.
            query: The search query.
            num_results: The number of URLs to collect.

        Returns:
            A list of ranked URLs.
        """
        max_results = max(num_results * 2, 5)
        results = await self.search_engine.run(query, max_results=max_results, as_string=False)
        if len(results) == 0:
            return []
        _results = "\n".join(f"{i}: {j}" for i, j in zip(range(max_results), results))
        prompt = COLLECT_AND_RANKURLS_PROMPT.format(topic=topic, query=query, results=_results)
        logger.debug(prompt)
        indices = await self._aask(prompt)
        try:
            indices = OutputParser.extract_struct(indices, list)
            assert all(isinstance(i, int) for i in indices)
        except Exception as e:
            logger.exception(f"fail to rank results for {e}")
            indices = list(range(max_results))
        results = [results[i] for i in indices]
        if self.rank_func:
            results = self.rank_func(results)
        return [i["link"] for i in results[:num_results]]

In [4]:
import tiktoken
model_name = "qwen2.5:7b"
# 假设 "qwen2.5:7b" 使用与 "gpt-4" 相同的编码器
encoding_name = "gpt-4"  # 或者其他已知与 "qwen2.5:7b" 兼容的编码器名称
encoding = tiktoken.get_encoding(encoding_name)

ValueError: Unknown encoding gpt-4. Plugins found: ['tiktoken_ext.openai_public']